In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# 1. Load dataset
data = pd.read_csv("data.csv")

# 2. Chọn feature và label
X = data[['queue_length', 'hour', 'day_of_week']]
y = data['wait_time']

# 3. Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Khởi tạo model Random Forest
model = RandomForestRegressor(
    n_estimators=100,   # số cây
    max_depth=10,       # chiều sâu tối đa để tránh overfit
    random_state=42
)

# 5. Train model
model.fit(X_train, y_train)

# 6. Dự đoán trên test set
y_pred = model.predict(X_test)

# 7. Đánh giá model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

print(f"MAE: {mae:.2f} phút")
print(f"RMSE: {rmse:.2f} phút")

# 8. Lưu model để dùng sau
joblib.dump(model, "random_forest_waiting_time.pkl")
print("Model đã được lưu thành công!")


MAE: 7.30 phút
RMSE: 10.36 phút
Model đã được lưu thành công!


In [2]:
import random
import pandas as pd
import joblib
from datetime import datetime

# Load model đã train
model = joblib.load("random_forest_waiting_time.pkl")

# Số vé test muốn tạo
num_tickets = 20  # ví dụ 20 vé
tickets = []

for i in range(num_tickets):
    # Sinh thời gian đến
    day = random.randint(1, 30)
    hour = random.randint(6, 20)
    minute = random.randint(0, 59)
    arrival_time = datetime(2025, 9, day, hour, minute)
    day_of_week = arrival_time.weekday()

    # Queue length
    if 9 <= hour <= 11 or 14 <= hour <= 16:
        queue_length = random.randint(5, 25)
    else:
        queue_length = random.randint(0, 10)

    tickets.append({
        "ticket_id": i+1,
        "arrival_time": arrival_time.strftime("%Y-%m-%d %H:%M"),
        "queue_length": queue_length,
        "hour": hour,
        "day_of_week": day_of_week
    })

# Chuyển sang DataFrame để dự đoán
df_tickets = pd.DataFrame(tickets)
X_test = df_tickets[['queue_length', 'hour', 'day_of_week']]
df_tickets['predicted_wait_time'] = model.predict(X_test)

# In kết quả
print(df_tickets)


    ticket_id      arrival_time  queue_length  hour  day_of_week  \
0           1  2025-09-18 08:43             1     8            3   
1           2  2025-09-15 14:33             5    14            0   
2           3  2025-09-26 08:40             4     8            4   
3           4  2025-09-10 06:57             3     6            2   
4           5  2025-09-10 07:16             5     7            2   
5           6  2025-09-13 08:51             0     8            5   
6           7  2025-09-17 09:12            21     9            2   
7           8  2025-09-18 16:20            22    16            3   
8           9  2025-09-07 19:25             2    19            6   
9          10  2025-09-11 13:30             3    13            3   
10         11  2025-09-28 16:32            19    16            6   
11         12  2025-09-23 09:48            19     9            1   
12         13  2025-09-09 11:57            23    11            1   
13         14  2025-09-13 09:14            24   